# RF6 Project Contributors

In [1]:
import json
import pandas as pd
from scripts.agora import agora_api
from scripts.farcaster import load_farcaster

# Process Farcaster and Badgeholders

In [2]:
#refresh_farcaster()
f = load_farcaster()

farcaster_users = pd.DataFrame(f)[['fid', 'username']].drop_duplicates().set_index('fid')['username'].to_dict()
farcaster_linked_addresses = pd.DataFrame(f)[['fid', 'address']].dropna().groupby('fid')['address'].apply(set).to_dict()
farcaster_address_to_id = pd.DataFrame(f)[['fid', 'address']].dropna().set_index('address')['fid'].to_dict()

In [3]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

badgeholder_attestation_data = load_json('data/attestations/599.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

Badgeholder attestations: 305


In [4]:
def convert_farcaster(farcaster_id):
    return int(farcaster_id['hex'], 16) if farcaster_id else None

badgeholders = []

for a in badgeholder_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['voterType'] == 'Guest':
        continue

    addr = a['recipient'].lower()
    fid = convert_farcaster(a['farcasterID'])
    username = farcaster_users.get(fid, '')
    badgeholders.append({
        'address': addr,
        'farcaster_id': fid,
        'farcaster_username': username,
        'source': 'attestation_schema_599'
    })
    linked_addresses = farcaster_linked_addresses.get(fid, [])
    for addr in linked_addresses:
        badgeholders.append({
            'address': addr.lower(),
            'farcaster_id': fid,
            'farcaster_username': username,
            'source': 'farcaster_linked_address'
        })

df_badgeholders = pd.DataFrame(badgeholders)
badgeholder_fids = set(df_badgeholders['farcaster_id'].unique())
df_badgeholders.tail(1)

,address,farcaster_id,farcaster_username,source
419,0xfb0a98121c3939657e59f835c66cfe20abb0cd4f,233135,krzkaczor,attestation_schema_599


In [5]:
rf6_projects = agora_api('retrofunding/rounds/6/projects')

Fetched 100 items, offset now at 100
Fetched a total of 142 items from retrofunding/rounds/6/projects.


In [6]:
df = pd.DataFrame(rf6_projects)
df['organization_name'] = df['organization'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)
df['team_fids'] = df['team'].apply(lambda team: [x['fid'] for x in team] if isinstance(team, list) else [])
df['num_badgeholders'] = df['team_fids'].apply(lambda x: len(set(x).intersection(badgeholder_fids)))
df['team_usernames'] = df['team'].apply(lambda team: [x['username'] for x in team] if isinstance(team, list) else [])
df['has_github'] = df['github'].apply(lambda x: len(x)>0)
df['has_contracts'] = df['contracts'].apply(lambda x: len(x)>0)
df['has_venture_funding'] = df['grantsAndFunding'].apply(lambda x: len(x.get('ventureFunding',[]))>0)
df['has_revenue'] = df['grantsAndFunding'].apply(lambda x: len(x.get('revenue',[]))>0)

def get_grants(g):
    lst = g.get('grants')
    total = 0
    if lst:
        for x in lst:
            a = x['amount']
            try:
                total += pd.to_numeric(a)
            except:
                a = 0
    return total

df['past_grants'] = df['grantsAndFunding'].apply(get_grants)
df['past_retrofunding'] = df['grantsAndFunding'].apply(lambda rf: sum([pd.to_numeric(x['amount']) for x in rf.get('retroFunding', [{'amount': 0}])]))

cols = [
    'applicationId', 'projectId', 'applicationCategory', 'organization_name', 'name',
    'has_github', 'has_contracts', 'has_venture_funding',
    'has_revenue', 'past_grants', 'past_retrofunding', 'team_usernames', 'num_badgeholders'
]

df_projects = df[cols].sort_values(by=['organization_name', 'name'])
df_projects.tail(1)

,applicationId,projectId,applicationCategory,organization_name,name,has_github,has_contracts,has_venture_funding,has_revenue,past_grants,past_retrofunding,team_usernames,num_badgeholders
18,0x28d507c6d2707ff3a7cd27232802d151bf429942b505...,0x016368b1434c826732ce14ef749ffc8f22a287e8845e...,GOVERNANCE_ANALYTICS,None,the1opus,False,False,False,False,0.0,0.0,[ecohealing],0


In [7]:
users = []
for app in rf6_projects:
    org = app.get('organization', {})
    org_name = org.get('name') if org else None
    team = app.get('team')
    if not team:
        continue
    for u in team:
        users.append({
            'fid': u['fid'],
            'username': u['username'],
            'organization_name': org_name,            
            'project_name': app['name'],
            'project_id': app['projectId'],
            'is_badgeholder': u['fid'] in badgeholder_fids
        })

df_users = pd.DataFrame(users).sort_values(by=['username', 'organization_name', 'project_name'])
df_users.tail(1)

,fid,username,organization_name,project_name,project_id,is_badgeholder
123,259525,zulfian666,None,Tadpole,0xe3dc2b3eea3c439d3662eab18bee971125398397a9eb...,False


In [8]:
df_projects.to_csv('data/rf6_project_applications.csv', index=0)
df_users.to_csv('data/rf6_users_by_project.csv', index=0)